<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Hamiltonian and momentum constraint equations using NRPyLaTeX 

## Author: Thiago Assumpção


## In this tutorial, we check that the UIUC metric satisfies the Hamiltonian and momentum constraint equations. We begin with the manual approach, and subsequently demonstrate how to reproduce the results using NRPyLaTeX.

**Notebook Status:** <font color='red'><b> Not validated </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation).

### NRPy+ Source Code for this module: [_____.py](../edit/_____.py)

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Initialize core Python/NRPy+ modules


1. [Step 8](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Initialize core Python/NRPy+ modules \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

In [1]:
import sympy as sp                       # SymPy: The Python computer algebra package upon which NRPy+ depends
import indexedexp as ixp                 # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import NRPy_param_funcs as par           # NRPy+: Parameter interface
from UnitTesting.assert_equal import check_zero # NRPy+: unit testing suite

In [2]:
!pip install nrpylatex~=1.2 > /dev/null
!pip freeze | grep nrpylatex

from nrpylatex import parse_latex
%load_ext nrpylatex.extension

nrpylatex==1.2.2


In [3]:
thismodule = __name__

<a id='uiucbh'></a>

# Step 2: UIUC metric \[Back to [top](#toc)\]
$$\label{uiucbh}$$

In [4]:
# All gridfunctions are written in terms of spherical coordinates (r, th, ph):
r, th, ph = sp.symbols('r th ph', real=True)

# Mass and spin parameters:
M, chi = sp.symbols("M chi", positive=True)

# Import UIUC Black Hole initial data
import BSSN.UIUCBlackHole as uibh
uibh.UIUCBlackHole(ComputeADMGlobalsOnly=True)
Sph_r_th_ph = [uibh.r,uibh.th,uibh.ph]
a   = uibh.a
rp  = uibh.rp
rm  = uibh.rm
rBL = uibh.rBL
SIG = uibh.SIG
DEL = uibh.DEL
AA  = uibh.AA
alphaSph    = uibh.alphaSph
betaSphU    = uibh.betaSphU
gammaSphDD  = uibh.gammaSphDD
KSphDD      = uibh.KSphDD

<a id='overwrite_lapse_shift'></a>

## Step 2a: Overwite lapse and shift compatible with Kerr metric  \[Back to [top](#toc)\]
$$\label{overwrite_lapse_shift}$$

The lapse and the non-vanishing component of the shift in spherical coordinates are given by (see Eqs. (6,7) in [Liu, Etienne, & Shapiro, PRD 80 121503, 2009](https://arxiv.org/abs/1001.4077)):

$$
\begin{aligned}
\alpha &= \sqrt{ \frac{\Delta \Sigma} {A} } \,, \\
\beta^{\phi} &= - \frac{ 2aMr_{\rm BL} } {A} \,.
\end{aligned}
$$

In [5]:
alphaSph    = sp.sqrt( (DEL*SIG) / AA )
betaSphU[2] = - (2*a*M*rBL) / AA

<a id='conformal_factor'></a>

## Step 2b: Define conformal factor, $\psi$, and its first and second derivatives \[Back to [top](#toc)\]
$$\label{conformal_factor}$$

The conformal factor is given by (see, e.g., Eq. (2) in [Liu, Etienne, & Shapiro, PRD 80 121503, 2009](https://arxiv.org/abs/1001.4077)):

$$
\psi^4 = \frac{A}{\Sigma} \,.
$$

In [6]:
# Define \psi^4
psi4 = AA/SIG
# Define \psi = (\psi^4)^(1/4)
psi = sp.Pow(psi4, sp.Rational(1,4))

In [7]:
# Define list with varibles for differentiation
# Since there is no \phi dependence, we only differentiate
# with respect to r and \theta
xSphU = [r, th]

# Compute first derivative of psi
psi_dD = ixp.zerorank1(DIM=3)
for i in range(2):
    psi_dD[i] = psi.diff(xSphU[i])

# Compute second derivative of psi
psi_dDD = ixp.zerorank2(DIM=3)
for i in range(2):
    for j in range(2):
        psi_dDD[i][j] = psi.diff(xSphU[i]).diff(xSphU[j])

<a id='nrpylatex'></a>

# Step 4: Constraint equations with NRPyLaTeX \[Back to [top](#toc)\]
$$\label{nrpylatex}$$

In [8]:
%%parse_latex --reset %%--debug

%% define LES coordinates
% coord [r, \theta, \phi]

%% define any relevant constant(s)
% define a M --const

%% initialize every component of the metric to zero
% define gammaDD --dim 3 --zero

%% initialize every component of the conformal metric to zero
% define gammatildeDD --dim 3 --zero

%% initialize extrinsic curvature tensor KDD
% define KDD --dim 3 --zero

%% initialize shift vector betaU
% define betaU --dim 3 --zero

%% initialize momentum constraint violation vector MU
% define mU --dim 3 --zero

%% define UIUC metric element
% srepl "rp"  -> "\text{rp}"
% srepl "rm"  -> "\text{rm}"
% srepl "r_{-}"  -> "\text{rp}"
% srepl "r_{+}"  -> "\text{rm}"
% srepl "rBL" -> "\text{rBL}"
% srepl "r_{BL}" -> "\text{rBL}"

% ignore "\begin{align}" "\end{align}"
\begin{align}
%% Define quantities that are used to construct the metric elements
r_{+}  &= M + \sqrt{M^2 - a^2} \\ %% Boyer-Lindquist outer horizon
r_{-}  &= M - \sqrt{M^2 - a^2} \\ %% Boyer-Lindquist inner horizon
r_{BL} &= r\left(1 + \frac{ r_{+} }{4r}\right)^2 \\
\Sigma &= r_{BL}^2 + a^2 \cos^2 \theta \\
\Delta &= r_{BL}^2 - 2 M r_{BL} + a^2 \\
A &= (r_{BL}^2 + a^2)^2 - \Delta a^2 \sin^2 \theta \\
%% Define metric elements
\gamma_{r r} &= \frac{ \Sigma (r + \frac{r_{+}}{4})^2}{ r^3 (r_{BL} - r_{-} )} \\ %% \gamma_{r r} = g_{0 0}
\gamma_{\theta\theta} &= \Sigma \\
\gamma_{\phi\phi} &= \frac{A \sin^2\theta}{\Sigma} \\
%% generate inverse gammaUU, determinant det(gammaDD), and connection GammaUDD
% assign gammaDD --metric
%% Define ADM variables
\alpha &= \sqrt{ \frac{\Delta \Sigma} {A} } \\
\beta^{\phi} &= - \frac{ 2aM r_{BL} } {A} \\
%% Define conformal factor
\psi &= \left(\frac{A}{\Sigma}\right)^{1/4} \\
%% Define conformal metric
\tilde{\gamma}_{ij} &= \psi^{{-4}} \gamma_{ij} \\
%% generate inverse gammatildeUU, determinant det(gammatildeDD), and connection GammatildeUDD
% assign gammatildeDD --metric
%% Define extrinsic curvature
K_{ij} &= \frac{1}{2\alpha} \left( \nabla_{i}\beta_{j} + \nabla_{j}\beta_{i} \right) \\
%% Trace of extrinsic curvature
K &= \gamma^{ij} K_{ij}  \\
%% Tracefree extrinsic curvature
A_{ij} &= K_{ij} - \frac{1}{3} K \gamma_{ij} \\
%% Conformal, tracefree extrinsic curvature
\tilde{A}_{ij} &= \psi^2 A_{ij} \\
%% Ricci tensor
\tilde{R}_{ij} &= -\frac{1}{2} \tilde{\gamma}^{kl} ( \tilde{\gamma}_{ij,lk} 
                  + \tilde{\gamma}_{kl,ji} - \tilde{\gamma}_{kj,li} - \tilde{\gamma}_{il,jk})
                  + \tilde{\gamma}^{kl} ( \tilde{\Gamma}^{m}_{il} \tilde{\Gamma}_{mkj} 
                  - \tilde{\Gamma}^{m}_{ij} \tilde{\Gamma}_{mkl}  ) \\
%% Ricci scalar
\tilde{R} &= \tilde{\gamma}^{ij} \tilde{R}_{ij} \\
%% Hamiltonian contraint violation
% srepl "\tilde{\nabla}^2" -> " \tilde{\gamma}^{ij} \tilde{\nabla}_i \tilde{\nabla}_j"
H &= \tilde{\nabla}^2 \psi - \frac{\psi \tilde{R}}{8} - \frac{ \psi^5 K^{{2}} }{12} 
   + \frac{\tilde{A}_{ij}\tilde{A}^{ij}}{8\psi^7} \\
%% Momentum constraint violation
%% srepl "\tilde{\nabla}^i" -> "\tilde{\gamma}^{ij} \tilde{\nabla}_j"
m^{i} &= \tilde{\nabla}_j \tilde{A}^{ij} - \frac{2}{3} \psi^6 \tilde{\gamma}^{ij} \tilde{\nabla}_j K
\end{align}


('A',
 'betaD',
 'RtildeDD',
 'r',
 'psi_cdtildeD_dD',
 'psi_cdtildeD',
 'KDD',
 'psi_cdtildeDD',
 'Delta',
 'GammatildeUDD',
 'alpha',
 'AtildeUU',
 'gammatildeDD',
 'H',
 'GammatildeDDD',
 'psi_dD',
 'rBL',
 'betaU',
 'K_cdtildeD',
 'psi_dDD',
 'theta',
 'a',
 'rm',
 'Sigma',
 'psi',
 'gammaDD',
 'rp',
 'GammaUDD',
 'betaD_cdD',
 'AtildeDD',
 'Rtilde',
 'AtildeUU_cdtildeD',
 'gammaUU',
 'epsilonUUU',
 'gammatildeUU',
 'ADD',
 'K',
 'M',
 'gammadet',
 'mU',
 'gammatildedet')

In [9]:
check_zero(mU[2])

True

In [13]:
check_zero(H)

True

<a id='latex_pdf_output'></a>

# Step 8: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-Hamiltonian-and-momentum-constraint-equations-using-NRPyLaTeX.pdf](Tutorial-Hamiltonian-and-momentum-constraint-equations-using-NRPyLaTeX.pdf). (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [11]:
# import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
# cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-Hamiltonian-and-momentum-constraint-equations-using-NRPyLaTeX")